In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [ ]:
#LIST_PATH
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

In [37]:
dataset[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/CT CT TAP 2.0 I30f 3',
 'CT',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/PT TEP TAP Corrig',
 'PT',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 '2017014',
 'cr_014_jul 25_2014.csv']

In [ ]:
#BUILD NIFTI FROM DATASET 1 2 et 3

json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_4/dataset_4.json'
dataset_4 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_4.append(info)

print(len(dataset_4))

#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', '78ca2911-9e9f-4f78-b80a-848024b95f92') 

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2', 'dataset', dataset)

In [40]:
#remove remove_bi_file

paths = get_series_path('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM')


In [43]:
from library_dicom.dicom_processor.tools.folders import *
for path in paths : 
    remove_bi_file(path)



In [39]:
#On met les erreurs de coté
#generer mip en meme temps 

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'



false_mask_dataset_1 = []
error_dataset_1 = []
path_mip = []

results_false_mask = []

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))
        subliste = []

        results = []

        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory+'/'+serie[-2], serie[]))
                #print("dicom effacé")

            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
                #print('dicom effacé')
            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)


0
MASK CORRECT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/PT p8273s0_FUE_ctac_resamp.img 3DRAMLA', 'PT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/CT', 'CT', '1.2.840.113704.1.111.920.1223892920.6', '2017031', 'cr_031_oct 13_2008.csv']
1
MASK CORRECT
EXPORT NIFTI PT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/CT CT TAP 3.0 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/PT PET TAP AC HD', 'PT', '1.2.840.113711.41813.3.11624.395627943.26.2116281012.16890', '2

In [45]:
print(len(error_dataset_1))
print(len(false_mask_dataset_1))

10
13


In [46]:
#SAVE THE SERIE WITH FALSE MASK 
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask', false_mask_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask_results', results_false_mask)

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'error', error_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP', 'path_mip', path_mip)

In [47]:
#create pdf 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#Si problème de bords pas pris en compte : generer nifti 

In [47]:

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print("dicom effacé")

            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print('dicom effacé')
            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)

0
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
5
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
10
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
11
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
12
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
13
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK


In [2]:
#POUR PET 2 ET PET 4 (SANS MASK)

#RELOAD JSON FROM PARSED DICOM IPYNB


json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4/GAINED_list_path_CT_PET4.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

534


In [4]:
#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', 'Elements') 

In [5]:
dataset[500]

['/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578',
 'CT',
 '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338',
 'PT',
 '1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006',
 '51011101881013']

In [ ]:
#CHECKER LES UNCONSTANT SPACING
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [7]:
#GENERATION NIFTI CT PET DES PET 2/4

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'

error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")



        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

2.840.113619.2.290.3.2831166708.745.1410849399.962/1.2.840.113619.2.290.3.2831166708.745.1410849399.967.3/1.2.840.113619.2.290.3.2831166708.745.1410849399.967.3', 'CT', '1.2.840.113619.2.290.3.2831166708.745.1410849399.962', '51011101731006']
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
91
EXPORT NIFTI PT
EXPORT NIFTI CT
92
EXPORT NIFTI PT
EXPORT NIFTI CT
93
EXPORT NIFTI PT
EXPORT NIFTI CT
94
EXPORT NIFTI PT
EXPORT NIFTI CT
95
EXPORT NIFTI PT
EXPORT NIFTI CT
96
EXPORT NIFTI PT
EXPORT NIFTI CT
97
EXPORT NIFTI PT
EXPORT NIFTI CT
98
EXPORT NIFTI PT
EXPORT NIFTI CT
99
EXPORT NIFTI PT
EXPORT NIFTI CT
100
EXPORT NIFTI PT
EXPORT NIFTI CT
101
EXPORT NIFTI PT
EXPORT NIFTI CT
102
EXPORT NIFTI PT
EXPORT NIFTI CT
103
EXPORT NIFTI PT
EXPORT NIFTI CT
104
EXPORT NIFTI PT
EXPORT NIFTI CT
105
EXPORT NIFTI PT
EXPORT NIFTI CT
106
EXPORT NIFTI PT
EXPORT NIFTI CT
107
EXPORT NIFTI PT
EXPORT NIFTI CT
108
EXPORT NIFTI PT
EXPORT NIFTI CT
109
EXPORT N